In [12]:
import os
import pandas as pd
import json
import openai
#from notebook_generator import generate_notebook

print(os.getcwd())

/Users/samuelalber/Documents/Research/jzou/salber/scAgent_v2


In [13]:
agent_output_path = "agent_analyses_unconditioned_gpt4o_p2.json"
csv_path_1 = os.path.join(home_dir, "gemini_2.5_pro_qa_conditioned.csv")
csv_path_2 = os.path.join(home_dir, "gemini_2.5_pro_qa_conditioned_p2.csv")
df_1 = pd.read_csv(csv_path_1)
df_2 = pd.read_csv(csv_path_2)
df = pd.concat([df_1, df_2])

df['analyses_full'] = df['analyses_full'].apply(eval)
analyses_full = df['analyses_full'].tolist()
home_dir = "/Users/samuelalber/Documents/Research/jzou/salber/scAgent_v2"

from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI()


In [14]:
def parse_response(response):
    if isinstance(response, str):
        response = re.sub(r'[\x00-\x1F\x7F]', '', response)
        response = response.replace('```json', '').replace('```python', '').replace('```', '').strip()
        parsed = json.loads(response)
    elif isinstance(response, list):
        parsed = []
        for r in response:
            r = r.replace('```json', '').replace('```python', '').replace('```', '').strip()
            r = json.loads(r)
            parsed.append(r)
    return parsed

In [15]:
analyses_overview = open(os.path.join(home_dir, "prompts", "DeepResearch_Analyses.txt")).read()


first_draft_prompt = f"""
You will be provided the background/introduction from a research paper.
The computational analyses done in the paper are hidden from you.

Your role is to propose a computational analysis that you think was most likely done in the paper.

Ensure that your output is in the specified JSON format.

For the analysis plan, think of the analysis plan as a scientific workflow:
    1. Start with exploratory data analysis that is broad and tests many things
    2. Then, focus on the more promising results from the exploratory phase by creating more focused analyses
    3. Include statistical validation of your results where appropiate
Do not number the analysis plan.
Each step in the analysis plan should be distinct from one another and could involve loading the data, conducting a statistical analysis, printing information about the AnnData object, etc.
Use however many steps is appropiate, but go for at least 5 steps. 


Ensure that your analyses solely use data explicitly mentioned in the paper. For example, if only RNA-seq data is mentioned, do NOT suggest spatial.
Likewise, if no spliced/unspliced RNA counts are mentioned, do NOT suggest RNA velocity.

Here are the previous analyses attempted:
{{past_analyses}}

Here is the background information from the paper:
{{paper_txt}}

For the rest of the prompt, we have examples of potential analyses:
{analyses_overview}
"""



system_prompt = f"""
You are a creative and skilled expert in single-cell transcriptomics computational analysis.

Output your response in the following JSON format (do not number the analysis steps, just list them):
{{
    "hypothesis": "...",
    "analysis_plan": ["First step", "Second step", ...],
    "summary": "A string describing the analysis in a detailed paragraph outlining how you will conduct the analysis"
}}
"""


critic_prompt = f"""
You will be given a hypothesis, analysis plan, and a summary of the analysis plan.
This analysis was generated by being given the background/introduction from a research paper (shown below).
The computational analyses done in the paper are hidden and the goal is to propose a computational analysis that is most likely to be in that hidden set.

Your role is to provide feedback for the analysis based on these goals.

Ensure that the analyses solely uses data explicitly mentioned in the paper. For example, if only RNA-seq data is mentioned, the analysis should NOT involve spatial analyses.
Likewise, if no spliced/unspliced RNA counts are mentioend, the analysis should NOT suggest RNA velocity.

Analysis Summary:
{{summary}}

Analysis Hypothesis:
{{hypothesis}}

Analysis Plan:
{{analysis_plan}}

Here is the background information from the paper:
{{paper_txt}}

Previous Analysis Attempted:
{{past_analyses}}

For the rest of the prompt, we have examples of potential analyses:
{analyses_overview}
"""

incorporate_prompt = f"""
You will be given a hypothesis, analysis plan, and a summary of the analysis plan.
This analysis was generated by being given the background/introduction from a research paper (shown below).
The computational analyses done in the paper are hidden and the goal is to propose a computational analysis that is most likely to be in that hidden set.

You will also be given feedback for the analysis in order so that it achieves these goals. 
Your role is to incorporate that feedback and update the analysis components (summary, hypothesis, analysis plan)

Analysis Summary:
{{summary}}

Analysis Hypothesis:
{{hypothesis}}

Analysis Plan:
{{analysis_plan}}

Feedback:
{{feedback}}

Here are the previous analyses attempted:
{{past_analyses}}

Here is the background information from the paper:
{{paper_txt}}

For the rest of the prompt, we have examples of potential analyses:
{analyses_overview}
"""

In [16]:
def get_response(prompt, system_prompt, use_json=False):
    if use_json:
        response = client.chat.completions.create(
            model="o3-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            response_format={"type": "json_object"}
        )
        result = response.choices[0].message.content
        return json.loads(result)
    else:
        response = client.chat.completions.create(
            model="o3-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
        )
        result = response.choices[0].message.content
        return result

In [ ]:
NUM_RUN = 5
for i in range(NUM_RUN):
    index = i + 1
    agent_output_path = os.path.join(home_dir, f"agent_analyses_unconditioned_gpt4o_{index}.json")
    all_agent_analyses = []
    for i, row in df.iterrows():
        context = row['context']
        analyses_full_paper = row['analyses_full']
        # Save context to temporary file
        temp_file = "temp_cond.txt"
        with open(temp_file, "w") as f:
            f.write(context)

        past_analyses = ""

        agent_analyses = []
        print(f"Running {len(analyses_full_paper)} analyses for paper {i+1}")
        for j in range(len(analyses_full_paper)):
            print(f"Running {j+1}/{len(analyses_full_paper)}")
            first_draft_prompt_filled = first_draft_prompt.format(past_analyses=past_analyses, paper_txt=context)
            first_draft = get_response(first_draft_prompt_filled, system_prompt, use_json=True)

            hypothesis, analysis_plan, summary = first_draft["hypothesis"], first_draft["analysis_plan"], first_draft["summary"]
            critic_prompt_filled = critic_prompt.format(summary=summary, hypothesis=hypothesis, analysis_plan=analysis_plan, 
                                                    paper_txt=context, past_analyses=past_analyses)
            feedback = get_response(critic_prompt_filled, "You are a single-cell bioinformatics expert providing feedback on code and analysis plan.", use_json=False)

            incorporate_prompt_filled = incorporate_prompt.format(summary=summary, hypothesis=hypothesis, analysis_plan=analysis_plan, 
                                                                    feedback=feedback, past_analyses=past_analyses, paper_txt=context)
            final_analysis = get_response(incorporate_prompt_filled, system_prompt, use_json=True)

            analysis = final_analysis['summary']
            agent_analyses.append(analysis)
            past_analyses += f"{analysis}\n\n"


        all_agent_analyses.append(agent_analyses)

        # Save intermediate result after each iteration
        with open(agent_output_path, "w") as f:
            json.dump(all_agent_analyses, f, indent=2)

        # Delete temporary file
        os.remove(temp_file)

Running 7 analyses for paper 1
Running 1/7
Running 2/7
Running 3/7
Running 4/7
Running 5/7
Running 6/7
Running 7/7
Running 6 analyses for paper 2
Running 1/6
Running 2/6
Running 3/6
Running 4/6
Running 5/6
Running 6/6
Running 11 analyses for paper 3
Running 1/11
Running 2/11
Running 3/11
Running 4/11
Running 5/11
Running 6/11
Running 7/11
Running 8/11
Running 9/11
Running 10/11
Running 11/11
Running 10 analyses for paper 4
Running 1/10
Running 2/10
Running 3/10
Running 4/10
Running 5/10
Running 6/10
Running 7/10
Running 8/10
Running 9/10
Running 10/10
Running 9 analyses for paper 5
Running 1/9
Running 2/9
Running 3/9
Running 4/9
Running 5/9
Running 6/9
Running 7/9
Running 8/9
Running 9/9
Running 16 analyses for paper 6
Running 1/16
Running 2/16
Running 3/16
Running 4/16
Running 5/16
Running 6/16
Running 7/16
Running 8/16
Running 9/16
Running 10/16
Running 11/16
Running 12/16
Running 13/16
Running 14/16
Running 15/16
Running 16/16
Running 12 analyses for paper 7
Running 1/12
Running 2/1

In [7]:
from dotenv import load_dotenv
load_dotenv()

import os
api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI(api_key=api_key)
with open(agent_output_path, "r") as f:
    all_agent_analyses = json.load(f)
all_judge_responses = []
for i, agent_analysis_list in enumerate(all_agent_analyses):
    judge_responses = []
    for j, agent_analysis in enumerate(agent_analysis_list):
        prompt = f"""
        You are given a proposed analysis for single-cell transcriptomics and a set of ground truth analyses. Your task is to determine whether the proposed analysis matches at least one analysis from the set of ground truth analyses.

        Proposed:
        {agent_analysis}

        Ground truth:
        {analyses_full[i]}

        Give your answer in the following format:
        {{
            "match": true/false,
            "reason": "explanation of the match or mismatch and if match, which analysis it matches"
        }}
        """
        response = client.responses.create(
            model="gpt-4o",
            input=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "input_text",
                            "text": prompt,
                        },
                    ]
                }
            ]
        )

        response = response.output_text
        judge_responses.append(response)
    all_judge_responses.append(judge_responses)

In [8]:
import numpy as np
df = pd.DataFrame({'raw_judge_response': all_judge_responses})

In [9]:
import re
def parse_response(response):
    def clean_and_parse(r):
        # Remove markdown fences and control characters
        r = re.sub(r'```(?:json|python)?', '', r, flags=re.IGNORECASE).strip()
        r = re.sub(r'[\x00-\x1F\x7F]', '', r)
        return json.loads(r)
    
    if isinstance(response, str):
        return clean_and_parse(response)
    elif isinstance(response, list):
        return [clean_and_parse(r) for r in response]
    else:
        raise TypeError("Unsupported input type for parse_response")

In [10]:
df['parsed_judge_response'] = df['raw_judge_response'].apply(parse_response)

In [11]:
df['num_match'] = df['parsed_judge_response'].apply(lambda x: sum(map(lambda y: int(y['match']), x)))
num_per_paper = df['parsed_judge_response'].apply(len)
micro_avg = (df['num_match'] / num_per_paper).mean()
macro_avg = df['num_match'].sum() / num_per_paper.sum()
micro_avg, macro_avg

(0.7193416305916306, 0.75)

In [19]:
df['num_match'] = df['parsed_judge_response'].apply(lambda x: sum(map(lambda y: int(y['match']), x)))
num_per_paper = df['parsed_judge_response'].apply(len)
micro_avg = (df['num_match'] / num_per_paper).mean()
macro_avg = df['num_match'].sum() / num_per_paper.sum()
micro_avg, macro_avg

(0.6115911865911866, 0.6813186813186813)

In [59]:
df['num_match'] = df['parsed_judge_response'].apply(lambda x: sum(map(lambda y: int(y['match']), x)))
num_per_paper = df['parsed_judge_response'].apply(len)
micro_avg = (df['num_match'] / num_per_paper).mean()
macro_avg = df['num_match'].sum() / num_per_paper.sum()
micro_avg, macro_avg

(0.4628110778110778, 0.42857142857142855)

In [17]:
df['num_match'] = df['parsed_judge_response'].apply(lambda x: sum(map(lambda y: int(y['match']), x)))
num_per_paper = df['parsed_judge_response'].apply(len)
micro_avg = (df['num_match'] / num_per_paper).mean()
macro_avg = df['num_match'].sum() / num_per_paper.sum()
micro_avg, macro_avg

(0.4142223054723055, 0.42857142857142855)